In [1]:
#The only thing to include in the paper/presentation from this code is the bar graph

import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
#import wfdb
import copy as cp
import scipy.signal as signal
import pickle
from sklearn import preprocessing
from tqdm import tqdm
import os
import re
import pandas as pd
import csv

from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from time import time
import timeit

import random
random.seed(1000)

In [2]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

#sns.set(rc={'figure.figsize':(11.7,8.27)})

C:\Users\bhoom\AppData\Local\Temp/ipykernel_39200/3159236473.py:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


# Side to side comparison plots

In [3]:
statallfeatures_dic = {
    'Run Time': [],
    'Accuracy': [],   ## To store the MEAN accuracy for 5-fold CV for each model
    'Standard Error': [],
    'Sensitivity': [],
    'Specificity': [],
    'Precision': [],
    'F1_Score': []   
}

In [4]:
## To store the accuracy for 5-fold CV for each model
indaccs_dic = {}

2017 train -> 2021 test

In [5]:
df1 = pd.read_csv("JORN17_dataset_v2.csv", na_values = '?').dropna()
df1['year'] = 2017
df2 = pd.read_csv("JORN21_dataset_v1.csv", na_values = '?').dropna()
df2['year'] = 2021
frames = [df1, df2]
df = pd.concat(frames)
df = df.reindex(columns=['OID_', 'Id', 'gridcode', 'Shape_Length', 'Shape_Area', 'CH_mean', 'ARVI_mean', 'ARVI_med', 'ARVI_max', 'EVI_mean', 'EVI_med', 'EVI_max', 'NDVI_mean', 'NDVI_med', 'NDVI_max', 'SAVI_mean', 'SAVI_med', 'SAVI_max', 'year', 'Veg_class'])
df


,OID_,Id,gridcode,Shape_Length,Shape_Area,CH_mean,ARVI_mean,ARVI_med,ARVI_max,EVI_mean,EVI_med,EVI_max,NDVI_mean,NDVI_med,NDVI_max,SAVI_mean,SAVI_med,SAVI_max,year,Veg_class
0,2,2,2,16.0,4.79,0.010000,0.223962,0.226896,0.239161,0.273236,0.274394,0.307885,0.402131,0.406066,0.427636,0.278782,0.280285,0.316995,2017,woody
1,3,3,3,8.6,2.24,0.010000,0.287822,0.287822,0.316384,0.302193,0.302193,0.305906,0.448325,0.448325,0.469259,0.301407,0.301407,0.304224,2017,woody
2,4,4,4,27.8,11.12,0.012500,0.289236,0.313074,0.459671,0.300200,0.318621,0.374641,0.447470,0.466219,0.570786,0.298798,0.314429,0.358302,2017,woody
3,5,5,5,24.2,10.62,0.060000,0.330255,0.375986,0.498250,0.313174,0.335601,0.386369,0.477596,0.517136,0.598966,0.309534,0.325886,0.368766,2017,woody
4,6,6,6,17.4,6.10,0.012000,0.275800,0.341526,0.416294,0.264112,0.280268,0.314841,0.431575,0.487791,0.518614,0.266352,0.273629,0.312146,2017,woody
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,2278,2278,2278,25.2,6.63,0.015000,0.128259,0.134121,0.167685,0.237659,0.245145,0.256763,0.334798,0.337580,0.365527,0.252164,0.257355,0.268680,2021,non-woody
2227,2279,2279,2279,15.2,5.33,0.134000,0.485618,0.489819,0.519026,0.416102,0.438885,0.441772,0.583201,0.585438,0.603520,0.386171,0.403764,0.410255,2021,woody
2228,2280,2280,2280,17.0,4.98,0.114000,0.313633,0.318283,0.474006,0.350407,0.341923,0.436805,0.483337,0.490012,0.591361,0.350956,0.346441,0.412744,2021,non-woody
2229,2281,2281,2281,19.8,6.49,0.011667,0.119920,0.108993,0.198284,0.230958,0.227932,0.271254,0.323523,0.317326,0.384514,0.243325,0.242532,0.278123,2021,non-woody


In [6]:
#Logistic Regression

kf = KFold(n_splits=5, random_state=3, shuffle=True)
model = LogisticRegression(solver='liblinear')

start_time = timeit.default_timer()

acc_score = []
Truth = []
Output = []

testindex8 = []
testindex9 = []

for train_index, test_index in kf.split(df): 
    
    X_train = df.iloc[train_index, 5:19]
    X_test = df.iloc[test_index, 5:19]
    Y_train = df.iloc[train_index, -1]
    Y_test = df.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)
    
    testindex8.append(test_index)
    testindex9.append(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print(sensitivity)
print(specificity)
print(precision)
print(f1_score)

statallfeatures_dic['Run Time'].append(elapsed)
statallfeatures_dic['Accuracy'].append(np.mean(acc_score))
statallfeatures_dic['Standard Error'].append(np.std(acc_score))
statallfeatures_dic['Sensitivity'].append(sensitivity)
statallfeatures_dic['Specificity'].append(specificity)
statallfeatures_dic['Precision'].append(precision)
statallfeatures_dic['F1_Score'].append(f1_score)

indaccs_dic['LogReg']=acc_score

---Run time is 0.12128859999999975 seconds ---

Accuracy of each fold: 
 [0.7390396659707724, 0.7463465553235908, 0.7630480167014614, 0.7546972860125261, 0.7599164926931107]
Avg accuracy: 0.7526096033402924
Std of accuracy : 
0.008832649603885677

[[ 944  731]
 [ 454 2661]]

              precision    recall  f1-score   support

   non-woody       0.68      0.56      0.61      1675
       woody       0.78      0.85      0.82      3115

    accuracy                           0.75      4790
   macro avg       0.73      0.71      0.72      4790
weighted avg       0.75      0.75      0.75      4790

0.5635820895522388
0.8542536115569823
0.6752503576537912
0.6143833387569151


In [7]:
woodyarea1 = 0
nonwoodyarea1 = 0
totalarea1 = 0

start_time = timeit.default_timer()

for i in testindex8:
    for j in i:
        totalarea1 += df.iat[j,4]
        if df.iat[j,-1] == "woody":
            woodyarea1 += df.iat[j,4]
        if df.iat[j,-1] == "non-woody":
            nonwoodyarea1 += df.iat[j,4]

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
needed = woodyarea1/totalarea1 * 100
print(needed)

---Run time is 0.5380897999999998 seconds ---
19.396377433556005


In [ ]:
awoodyarea4 = 0
anonwoodyarea4 = 0
atotalarea4 = 0

start_time = timeit.default_timer()

for i in testindex8:
    for j in i:
        for k in range(0,1):
            for l in testindex9[k]:
                if l == "woody":
                    awoodyarea4 += df.iat[j,4]
                    atotalarea4 += df.iat[j,4]
                if l == "non-woody":
                    anonwoodyarea4 += df.iat[j,4]
                    atotalarea4 += df.iat[j,4]
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
aFWC4 = awoodyarea4/atotalarea4 * 100
                    
print(aFWC4)

In [ ]:
bwoodyarea4 = 0
bnonwoodyarea4 = 0
btotalarea4 = 0

start_time = timeit.default_timer()

for i in testindex8:
    for j in i:
        for k in range(1,2):
            for l in testindex9[k]:
                if l == "woody":
                    bwoodyarea4 += df.iat[j,4]
                    btotalarea4 += df.iat[j,4]
                if l == "non-woody":
                    bnonwoodyarea4 += df.iat[j,4]
                    btotalarea4 += df.iat[j,4]
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
bFWC4 = bwoodyarea4/btotalarea4 * 100
                    
print(bFWC4)

In [ ]:
cwoodyarea4 = 0
cnonwoodyarea4 = 0
ctotalarea4 = 0

start_time = timeit.default_timer()

for i in testindex8:
    for j in i:
        for k in range(2,3):
            for l in testindex9[k]:
                if l == "woody":
                    cwoodyarea4 += df.iat[j,4]
                    ctotalarea4 += df.iat[j,4]
                if l == "non-woody":
                    cnonwoodyarea4 += df.iat[j,4]
                    ctotalarea4 += df.iat[j,4]
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
cFWC4 = cwoodyarea4/ctotalarea4 * 100
                    
print(cFWC4)

In [ ]:
dwoodyarea4 = 0
dnonwoodyarea4 = 0
dtotalarea4 = 0

start_time = timeit.default_timer()

for i in testindex8:
    for j in i:
        for k in range(3,4):
            for l in testindex9[k]:
                if l == "woody":
                    dwoodyarea4 += df.iat[j,4]
                    dtotalarea4 += df.iat[j,4]
                if l == "non-woody":
                    dnonwoodyarea4 += df.iat[j,4]
                    dtotalarea4 += df.iat[j,4]
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
dFWC4 = dwoodyarea4/dtotalarea4 * 100
                    
print(dFWC4)

In [ ]:
ewoodyarea4 = 0
enonwoodyarea4 = 0
etotalarea4 = 0

start_time = timeit.default_timer()

for i in testindex8:
    for j in i:
        for k in range(4,5):
            for l in testindex9[k]:
                if l == "woody":
                    ewoodyarea4 += df.iat[j,4]
                    etotalarea4 += df.iat[j,4]
                if l == "non-woody":
                    enonwoodyarea4 += df.iat[j,4]
                    etotalarea4 += df.iat[j,4]
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
eFWC4 = ewoodyarea4/etotalarea4 * 100
                    
print(eFWC4)

In [ ]:
tFWC4 = (aFWC4 +bFWC4 +cFWC4 +dFWC4 +eFWC4) / 5
print(tFWC4)

In [ ]:
#LDA
kf = KFold(n_splits=5, random_state=3, shuffle=True)
model=LinearDiscriminantAnalysis()

acc_score = []
Truth = []
Output = []

testindex1 = []
testindex2 = []
  
for train_index, test_index in kf.split(df): 
    X_train = df.iloc[train_index, 5:19]
    X_test = df.iloc[test_index, 5:19]
    Y_train = df.iloc[train_index, -1]
    Y_test = df.iloc[test_index, -1]


    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)
    
    testindex1.append(test_index)
    testindex2.append(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print(sensitivity)
print(specificity)
print(precision)
print(f1_score)

statallfeatures_dic['Run Time'].append(elapsed)
statallfeatures_dic['Accuracy'].append(np.mean(acc_score))
statallfeatures_dic['Standard Error'].append(np.std(acc_score))
statallfeatures_dic['Sensitivity'].append(sensitivity)
statallfeatures_dic['Specificity'].append(specificity)
statallfeatures_dic['Precision'].append(precision)
statallfeatures_dic['F1_Score'].append(f1_score)

indaccs_dic['LDA']=acc_score

In [ ]:
woodyarea1 = 0
nonwoodyarea1 = 0
totalarea1 = 0

start_time = timeit.default_timer()

for i in testindex1:
    for j in i:
        totalarea1 += df.iat[j,4]
        if df.iat[j,-1] == "woody":
            woodyarea1 += df.iat[j,4]
        if df.iat[j,-1] == "non-woody":
            nonwoodyarea1 += df.iat[j,4]

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
print(woodyarea1/totalarea1 * 100)

needed2 = woodyarea1/totalarea1 * 100
print(needed2)

In [ ]:
awoodyarea1 = 0
anonwoodyarea1 = 0
atotalarea1 = 0
start_time = timeit.default_timer()

for i in testindex1:
    for j in i:
        for k in range(0,1):
            for l in testindex2[k]:
                if l == "woody":
                    awoodyarea1 += df.iat[j,4]
                    atotalarea1 += df.iat[j,4]
                if l == "non-woody":
                    anonwoodyarea1 += df.iat[j,4]
                    atotalarea1 += df.iat[j,4]

                    
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
aFWC = awoodyarea1/atotalarea1 * 100
                    
print(aFWC)

In [ ]:
bwoodyarea1 = 0
bnonwoodyarea1 = 0
btotalarea1 = 0
start_time = timeit.default_timer()

for i in testindex1:
    for j in i:
        for k in range(1,2):
            for l in testindex2[k]:
                if l == "woody":
                    bwoodyarea1 += df.iat[j,4]
                    btotalarea1 += df.iat[j,4]
                if l == "non-woody":
                    bnonwoodyarea1 += df.iat[j,4]
                    btotalarea1 += df.iat[j,4]

                    
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
bFWC = bwoodyarea1/btotalarea1 * 100
                    
print(bFWC)

In [ ]:
cwoodyarea1 = 0
cnonwoodyarea1 = 0
ctotalarea1 = 0
start_time = timeit.default_timer()

for i in testindex1:
    for j in i:
        for k in range(2,3):
            for l in testindex2[k]:
                if l == "woody":
                    cwoodyarea1 += df.iat[j,4]
                    ctotalarea1 += df.iat[j,4]
                if l == "non-woody":
                    cnonwoodyarea1 += df.iat[j,4]
                    ctotalarea1 += df.iat[j,4]

                    
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
cFWC = cwoodyarea1/ctotalarea1 * 100
                    
print(cFWC)

In [ ]:
dwoodyarea1 = 0
dnonwoodyarea1 = 0
dtotalarea1 = 0
start_time = timeit.default_timer()

for i in testindex1:
    for j in i:
        for k in range(3,4):
            for l in testindex2[k]:
                if l == "woody":
                    dwoodyarea1 += df.iat[j,4]
                    dtotalarea1 += df.iat[j,4]
                if l == "non-woody":
                    dnonwoodyarea1 += df.iat[j,4]
                    dtotalarea1 += df.iat[j,4]

                    
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
dFWC = dwoodyarea1/dtotalarea1 * 100
                    
print(dFWC)

In [ ]:
ewoodyarea1 = 0
enonwoodyarea1 = 0
etotalarea1 = 0
start_time = timeit.default_timer()

for i in testindex1:
    for j in i:
        for k in range(4,5):
            for l in testindex2[k]:
                if l == "woody":
                    ewoodyarea1 += df.iat[j,4]
                    etotalarea1 += df.iat[j,4]
                if l == "non-woody":
                    enonwoodyarea1 += df.iat[j,4]
                    etotalarea1 += df.iat[j,4]

                    
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
eFWC = ewoodyarea1/etotalarea1 * 100
                    
print(eFWC)

In [ ]:
tFWC = (aFWC + bFWC + cFWC + dFWC + eFWC) / 5
print(tFWC)

In [ ]:
#QDA
kf = KFold(n_splits=5, random_state=3, shuffle=True)
model=QuadraticDiscriminantAnalysis()

acc_score = []
Truth = []
Output = []

testindex3 = []
testindex4 = []


for train_index, test_index in kf.split(df): 
    X_train = df.iloc[train_index, 5:19]
    X_test = df.iloc[test_index, 5:19]
    Y_train = df.iloc[train_index, -1]
    Y_test = df.iloc[test_index, -1]


    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)


    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)
    
    testindex3.append(test_index)
    testindex4.append(pred_values)

elapsed = timeit.default_timer() - start_time


print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print(sensitivity)
print(specificity)
print(precision)
print(f1_score)

statallfeatures_dic['Run Time'].append(elapsed)
statallfeatures_dic['Accuracy'].append(np.mean(acc_score))
statallfeatures_dic['Standard Error'].append(np.std(acc_score))
statallfeatures_dic['Sensitivity'].append(sensitivity)
statallfeatures_dic['Specificity'].append(specificity)
statallfeatures_dic['Precision'].append(precision)
statallfeatures_dic['F1_Score'].append(f1_score)

indaccs_dic['QDA']=acc_score

In [ ]:
woodyarea1 = 0
nonwoodyarea1 = 0
totalarea1 = 0

start_time = timeit.default_timer()

for i in testindex3:
    for j in i:
        totalarea1 += df.iat[j,4]
        if df.iat[j,-1] == "woody":
            woodyarea1 += df.iat[j,4]
        if df.iat[j,-1] == "non-woody":
            nonwoodyarea1 += df.iat[j,4]

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
needed3 = woodyarea1/totalarea1 * 100
print(needed3)

In [ ]:
awoodyarea2 = 0
anonwoodyarea2 = 0
atotalarea2 = 0
start_time = timeit.default_timer()

for i in testindex3:
    for j in i:
        for k in range(0,1):
            for l in testindex4[k]:
                if l == "woody":
                    awoodyarea2 += df.iat[j,4]
                    atotalarea2 += df.iat[j,4]
                if l == "non-woody":
                    anonwoodyarea2 += df.iat[j,4]
                    atotalarea2 += df.iat[j,4]
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
aFWC2 = awoodyarea2/atotalarea2 * 100
                    
print(aFWC2)

In [ ]:
bwoodyarea2 = 0
bnonwoodyarea2 = 0
btotalarea2 = 0
start_time = timeit.default_timer()

for i in testindex3:
    for j in i:
        for k in range(1,2):
            for l in testindex4[k]:
                if l == "woody":
                    bwoodyarea2 += df.iat[j,4]
                    btotalarea2 += df.iat[j,4]
                if l == "non-woody":
                    bnonwoodyarea2 += df.iat[j,4]
                    btotalarea2 += df.iat[j,4]
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
bFWC2 = bwoodyarea2/btotalarea2 * 100
                    
print(bFWC2)

In [ ]:
cwoodyarea2 = 0
cnonwoodyarea2 = 0
ctotalarea2 = 0
start_time = timeit.default_timer()

for i in testindex3:
    for j in i:
        for k in range(2,3):
            for l in testindex4[k]:
                if l == "woody":
                    cwoodyarea2 += df.iat[j,4]
                    ctotalarea2 += df.iat[j,4]
                if l == "non-woody":
                    cnonwoodyarea2 += df.iat[j,4]
                    ctotalarea2 += df.iat[j,4]
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
cFWC2 = cwoodyarea2/ctotalarea2 * 100
                    
print(cFWC2)

In [ ]:
dwoodyarea2 = 0
dnonwoodyarea2 = 0
dtotalarea2 = 0
start_time = timeit.default_timer()

for i in testindex3:
    for j in i:
        for k in range(3,4):
            for l in testindex4[k]:
                if l == "woody":
                    dwoodyarea2 += df.iat[j,4]
                    dtotalarea2 += df.iat[j,4]
                if l == "non-woody":
                    dnonwoodyarea2 += df.iat[j,4]
                    dtotalarea2 += df.iat[j,4]
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
dFWC2 = dwoodyarea2/dtotalarea2 * 100
                    
print(dFWC2)

In [ ]:
ewoodyarea2 = 0
enonwoodyarea2 = 0
etotalarea2 = 0
start_time = timeit.default_timer()

for i in testindex3:
    for j in i:
        for k in range(4,5):
            for l in testindex4[k]:
                if l == "woody":
                    ewoodyarea2 += df.iat[j,4]
                    etotalarea2 += df.iat[j,4]
                if l == "non-woody":
                    enonwoodyarea2 += df.iat[j,4]
                    etotalarea2 += df.iat[j,4]
elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
eFWC2 = ewoodyarea2/etotalarea2 * 100
                    
print(eFWC2)

In [ ]:
tFWC2 = (aFWC2 + bFWC2 + cFWC2 + dFWC2 + eFWC2) / 5

#KNN-CV
KNN_result = []
X=subject_df.iloc[:, 2:17]
Y=subject_df.iloc[:, -1]
crossvalidation = KFold(n_splits=5, random_state=3, shuffle=True)
KNNResult = []
for k in range(1,26):
    knn = neighbors.KNeighborsClassifier(n_neighbors=k)
    model = knn.fit(X, Y)
    scores=cross_val_score(model, X, Y, cv=crossvalidation)
    print('K={}'.format(k))
    print()
    print('Accuracy of each fold: \n {}'.format(scores))
    print()
    print("Avg accuracy: {}".format(scores.mean()))
    print()
    KNN_result.append(scores.mean())
print(KNN_result)

In [ ]:
KNN_result = []
X=df.iloc[:, 5:19]
Y=df.iloc[:, -1]
crossvalidation = KFold(n_splits=5, random_state=3, shuffle=True)
KNNResult = []
for k in range(1,11):
    knn = neighbors.KNeighborsClassifier(n_neighbors=k)
    model = knn.fit(X, Y)
    scores=cross_val_score(model, X, Y, cv=crossvalidation)
    print('K={}'.format(k))
    print()
    print('Accuracy of each fold: \n {}'.format(scores))
    print()
    print("Avg accuracy: {}".format(scores.mean()))
    print()
    KNN_result.append(scores.mean())
print(KNN_result)

In [ ]:
#KNN-CV
kf = KFold(n_splits=5, random_state=3, shuffle=True)
model=neighbors.KNeighborsClassifier(n_neighbors=9)

acc_score = []
Truth = []
Output = []

testindex5 = []
testindex6 = []

for train_index, test_index in kf.split(df): 
    X_train = df.iloc[train_index, 5:19]
    X_test = df.iloc[test_index, 5:19]
    Y_train = df.iloc[train_index, -1]
    Y_test = df.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)
    
    testindex5.append(test_index)
    testindex6.append(pred_values)

elapsed = timeit.default_timer() - start_time

print()
print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print(sensitivity)
print(specificity)
print(precision)
print(f1_score)

statallfeatures_dic['Run Time'].append(elapsed)
statallfeatures_dic['Accuracy'].append(np.mean(acc_score))
statallfeatures_dic['Standard Error'].append(np.std(acc_score))
statallfeatures_dic['Sensitivity'].append(sensitivity)
statallfeatures_dic['Specificity'].append(specificity)
statallfeatures_dic['Precision'].append(precision)
statallfeatures_dic['F1_Score'].append(f1_score)

indaccs_dic['KNN']=acc_score

In [ ]:
woodyarea1 = 0
nonwoodyarea1 = 0
totalarea1 = 0

start_time = timeit.default_timer()

for i in testindex5:
    for j in i:
        totalarea1 += df.iat[j,4]
        if df.iat[j,-1] == "woody":
            woodyarea1 += df.iat[j,4]
        if df.iat[j,-1] == "non-woody":
            nonwoodyarea1 += df.iat[j,4]

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
needed4 = woodyarea1/totalarea1 * 100
print(needed4)

In [ ]:
awoodyarea3 = 0
anonwoodyarea3 = 0
atotalarea3 = 0

start_time = timeit.default_timer()

for i in testindex5:
    for j in i:
        for k in range(0,1):
            for l in testindex6[k]:
                if l == "woody":
                    awoodyarea3 += df.iat[j,4]
                    atotalarea3 += df.iat[j,4]
                if l == "non-woody":
                    anonwoodyarea3 += df.iat[j,4]
                    atotalarea3 += df.iat[j,4]

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
aFWC3 = awoodyarea3/atotalarea3 * 100
                    
print(aFWC3)

In [ ]:
bwoodyarea3 = 0
bnonwoodyarea3 = 0
btotalarea3 = 0

start_time = timeit.default_timer()

for i in testindex5:
    for j in i:
        for k in range(1,2):
            for l in testindex6[k]:
                if l == "woody":
                    bwoodyarea3 += df.iat[j,4]
                    btotalarea3 += df.iat[j,4]
                if l == "non-woody":
                    bnonwoodyarea3 += df.iat[j,4]
                    btotalarea3 += df.iat[j,4]

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
bFWC3 = bwoodyarea3/btotalarea3 * 100
                    
print(bFWC3)

In [ ]:
cwoodyarea3 = 0
cnonwoodyarea3 = 0
ctotalarea3 = 0

start_time = timeit.default_timer()

for i in testindex5:
    for j in i:
        for k in range(2,3):
            for l in testindex6[k]:
                if l == "woody":
                    cwoodyarea3 += df.iat[j,4]
                    ctotalarea3 += df.iat[j,4]
                if l == "non-woody":
                    cnonwoodyarea3 += df.iat[j,4]
                    ctotalarea3 += df.iat[j,4]

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
cFWC3 = cwoodyarea3/ctotalarea3 * 100
                    
print(cFWC3)

In [ ]:
dwoodyarea3 = 0
dnonwoodyarea3 = 0
dtotalarea3 = 0

start_time = timeit.default_timer()

for i in testindex5:
    for j in i:
        for k in range(3,4):
            for l in testindex6[k]:
                if l == "woody":
                    dwoodyarea3 += df.iat[j,4]
                    dtotalarea3 += df.iat[j,4]
                if l == "non-woody":
                    dnonwoodyarea3 += df.iat[j,4]
                    dtotalarea3 += df.iat[j,4]

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
dFWC3 = dwoodyarea3/dtotalarea3 * 100
                    
print(dFWC3)

In [ ]:
ewoodyarea3 = 0
enonwoodyarea3 = 0
etotalarea3 = 0

start_time = timeit.default_timer()

for i in testindex5:
    for j in i:
        for k in range(4,5):
            for l in testindex6[k]:
                if l == "woody":
                    ewoodyarea3 += df.iat[j,4]
                    etotalarea3 += df.iat[j,4]
                if l == "non-woody":
                    enonwoodyarea3 += df.iat[j,4]
                    etotalarea3 += df.iat[j,4]

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)
eFWC3 = ewoodyarea3/etotalarea3 * 100
                    
print(eFWC3)

In [ ]:
tFWC3 = (aFWC3 + bFWC3 + cFWC3 + dFWC3 + eFWC3) / 5

In [ ]:
transitionsdf = pd.DataFrame(
    data = np.transpose(np.array([
                                statallfeatures_dic["Accuracy"],
                                statallfeatures_dic["Standard Error"],
                                statallfeatures_dic["Sensitivity"],
                                statallfeatures_dic["Specificity"],
                                statallfeatures_dic["Precision"],
                                statallfeatures_dic["F1_Score"],
                                statallfeatures_dic["Run Time"] 
                            ])
                       ),
    
    columns = ["Accuracy", "Standard Error", "Sensitivity", "Specificity", "Precision", "F1-Score", "Run Time"]
    
)

transitionsdf.set_index(pd.Index(["LogReg", "LDA", "QDA", "KNN-CV"]))

In [ ]:
#Bar Graph
models = ['LogReg', 'LDA', 'QDA', 'KNN-CV']
print(models)
accuracies = statallfeatures_dic["Accuracy"]
print("Accuracies" + str(accuracies))
sensitivities = statallfeatures_dic["Sensitivity"]
print("Sen" + str(sensitivities))
specificities = statallfeatures_dic["Specificity"]
print("Spec" + str(specificities))
precisions = statallfeatures_dic["Precision"]
print("P" + str(precisions))
f1_scores = statallfeatures_dic["F1_Score"]
print("F1" + str(f1_scores))

x = 6*np.arange(len(models))
width = 0.8

fig, ax = plt.subplots(figsize=(12, 6))
import seaborn as sns
colors = sns.color_palette("BrBG")

acc = ax.bar(x-width*2, accuracies, width, label='Accuracy', color=colors[0])
sen = ax.bar(x-width, sensitivities, width, label='Sensitivity', color=colors[1])
spe = ax.bar(x,specificities, width, label='Specificity', color=colors[2])
pre = ax.bar(x+width, precisions, width, label='Precision', color=colors[3])
f1 = ax.bar(x+width*2 , f1_scores, width, label='F1_Score', color=colors[4])

ax.set_xlabel('Classification Model', fontsize=14)
ax.set_ylabel('Percentage', fontsize=14)
ax.set_title('Model Performance For Predicting Woody and Nonwoody Plants Using All Features', fontsize=16)
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()
ax.set(ylim=(0.0, 1))


fig.tight_layout()

plt.legend(loc="upper left", prop={'size': 10})
plt.show()

fig.savefig('AllFeaturesPerformance.png')

#Code from https://matplotlib.org/stable/gallery/lines_bars_and_markers/barchart.html#sphx-glr-gallery-lines-bars-and-markers-barchart-py

In [ ]:
#Box Plot Accuracies

models = ['LogReg', 'LDA', 'QDA', 'KNN-CV']

x = np.arange(len(models))+1

LogRegAcc = indaccs_dic['LogReg']
LDAAcc = indaccs_dic['LDA']
QDAAcc = indaccs_dic['QDA']
KNNAcc = indaccs_dic['KNN']

data = [LogRegAcc, LDAAcc, QDAAcc, KNNAcc]

fig, ax = plt.subplots(figsize = (15,7.5))
bplot = ax.boxplot(data, patch_artist = True)

ax.set_title('Accuracies of AFIB detection Using All Features for Various Classifiers', fontsize=15)
ax.set_xlabel('Classification Model', fontsize=15)
ax.set_ylabel('Accuracy', fontsize=15)
ax.set_xticks(x)
ax.set_xticklabels(models)

import seaborn as sns
#colors = sns.color_palette("colorblind")
colors = sns.color_palette()

for patch, color in zip(bplot['boxes'], colors):
        patch.set_facecolor(color)
plt.show()

In [ ]:
print("Log reg act: " + str(needed))
print("Log reg pred: " + str(tFWC4))
print("QDA act: " + str(needed2))
print("QDA pred: " + str(tFWC2))
print("LDA act: " + str(needed3))
print("LDA pred: " + str(tFWC))
print("KNN act: " + str(needed4))
print("KNN pred: " +str(tFWC3))

In [ ]:
Logerrorrate = abs((needed-tFWC4)/needed) *100
QDAerrorrate = abs((needed-tFWC2)/needed) *100
LDAerrorrate = abs((needed-tFWC)/needed) *100
KNNerrorrate = abs((needed-tFWC3)/needed) *100

print("log ER = " + str(Logerrorrate))
print("QDA ER = " + str(QDAerrorrate))
print("LDA ER = " + str(LDAerrorrate))
print("KNN ER = " + str(KNNerrorrate))